# AUT64 Brute Force version CPU

Auteur: [Kaci Amaouche](mailto:amaouchekaci28@gmail.com)

Dans ce [Jupyter](https://jupyter.org/) notebook,nous présentons une implémentation d'une attaque type bruteforce de AUT64 sur CPU en utilisant Python. Si vous n'êtes pas familier avec Jupyter, vous pouvez jeter un coup d'œil rapide à la documentation ou aux tutoriels disponibles[Notebook Basics](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Notebook%20Basics.html) guide (~5min).

Ce notebook comporte:

* [Configuration de l'environnement]
* [Implémentation]

## 1 - Configuration de l'environnement

Pour pouvoir exécuter ces scripts, il va falloir:
1. Ce notebook (the `.ipynb` file)
1. Python >= 3.8


## 2 - Implémentation

On va commencer par importer les librairies nécessaires

In [26]:
import itertools
import copy
from copy import deepcopy
from itertools import permutations
import math

Maintenant, on définit les constates nécessaires pour le système, notamment les tables de recherches table_ln, table_un, table_offset et table_sub et la table TD qui est essentiel pour reconstituer la clé KG

In [4]:
table_ln = [0x4, 0x5, 0x6, 0x7, 0x0, 0x1, 0x2, 0x3, # Round 0
            0x5, 0x4, 0x7, 0x6, 0x1, 0x0, 0x3, 0x2, # Round 1
            0x6, 0x7, 0x4, 0x5, 0x2, 0x3, 0x0, 0x1, # ...
            0x7, 0x6, 0x5, 0x4, 0x3, 0x2, 0x1, 0x0, 
            0x0, 0x1, 0x2, 0x3, 0x4, 0x5, 0x6, 0x7,
            0x1, 0x0, 0x3, 0x2, 0x5, 0x4, 0x7, 0x6,
            0x2, 0x3, 0x0, 0x1, 0x6, 0x7, 0x4, 0x5,
            0x3, 0x2, 0x1, 0x0, 0x7, 0x6, 0x5, 0x4,
            0x5, 0x4, 0x7, 0x6, 0x1, 0x0, 0x3, 0x2,
            0x4, 0x5, 0x6, 0x7, 0x0, 0x1, 0x2, 0x3,
            0x7, 0x6, 0x5, 0x4, 0x3, 0x2, 0x1, 0x0, 
            0x6, 0x7, 0x4, 0x5, 0x2, 0x3, 0x0, 0x1,
            0x1, 0x0, 0x3, 0x2, 0x5, 0x4, 0x7, 0x6,
            0x0, 0x1, 0x2, 0x3, 0x4, 0x5, 0x6, 0x7,
            0x3, 0x2, 0x1, 0x0, 0x7, 0x6, 0x5, 0x4,
            0x2, 0x3, 0x0, 0x1, 0x6, 0x7, 0x4, 0x5,
            0x6, 0x7, 0x4, 0x5, 0x2, 0x3, 0x0, 0x1,
            0x7, 0x6, 0x5, 0x4, 0x3, 0x2, 0x1, 0x0,
            0x4, 0x5, 0x6, 0x7, 0x0, 0x1, 0x2, 0x3,
            0x5, 0x4, 0x7, 0x6, 0x1, 0x0, 0x3, 0x2,
            0x2, 0x3, 0x0, 0x1, 0x6, 0x7, 0x4, 0x5,
            0x3, 0x2, 0x1, 0x0, 0x7, 0x6, 0x5, 0x4,
            0x0, 0x1, 0x2, 0x3, 0x4, 0x5, 0x6, 0x7,
            0x1, 0x0, 0x3, 0x2, 0x5, 0x4, 0x7, 0x6] # Round 24

table_un = [0x1, 0x0, 0x3, 0x2, 0x5, 0x4, 0x7, 0x6, # Round 0
            0x0, 0x1, 0x2, 0x3, 0x4, 0x5, 0x6, 0x7, # Round 1
            0x3, 0x2, 0x1, 0x0, 0x7, 0x6, 0x5, 0x4, # ...
            0x2, 0x3, 0x0, 0x1, 0x6, 0x7, 0x4, 0x5, 
            0x5, 0x4, 0x7, 0x6, 0x1, 0x0, 0x3, 0x2, 
            0x4, 0x5, 0x6, 0x7, 0x0, 0x1, 0x2, 0x3, 
            0x7, 0x6, 0x5, 0x4, 0x3, 0x2, 0x1, 0x0, 
            0x6, 0x7, 0x4, 0x5, 0x2, 0x3, 0x0, 0x1, 
            0x3, 0x2, 0x1, 0x0, 0x7, 0x6, 0x5, 0x4,
            0x2, 0x3, 0x0, 0x1, 0x6, 0x7, 0x4, 0x5, 
            0x1, 0x0, 0x3, 0x2, 0x5, 0x4, 0x7, 0x6, 
            0x0, 0x1, 0x2, 0x3, 0x4, 0x5, 0x6, 0x7,
            0x7, 0x6, 0x5, 0x4, 0x3, 0x2, 0x1, 0x0,
            0x6, 0x7, 0x4, 0x5, 0x2, 0x3, 0x0, 0x1,
            0x5, 0x4, 0x7, 0x6, 0x1, 0x0, 0x3, 0x2,
            0x4, 0x5, 0x6, 0x7, 0x0, 0x1, 0x2, 0x3,
            0x2, 0x3, 0x0, 0x1, 0x6, 0x7, 0x4, 0x5,
            0x3, 0x2, 0x1, 0x0, 0x7, 0x6, 0x5, 0x4,
            0x0, 0x1, 0x2, 0x3, 0x4, 0x5, 0x6, 0x7,
            0x1, 0x0, 0x3, 0x2, 0x5, 0x4, 0x7, 0x6,
            0x6, 0x7, 0x4, 0x5, 0x2, 0x3, 0x0, 0x1,
            0x7, 0x6, 0x5, 0x4, 0x3, 0x2, 0x1, 0x0,
            0x4, 0x5, 0x6, 0x7, 0x0, 0x1, 0x2, 0x3,
            0x5, 0x4, 0x7, 0x6, 0x1, 0x0, 0x3, 0x2] # Round 24
# AUT64 la table T_offset pour la fonction de compression         
# octet d'entrée:   0    1    2    3    4    5    6    7    8    9    A    B    C    D    E    F  | octet de clé               
table_offset = [ 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, # 0
                 0x0, 0x1, 0x2, 0x3, 0x4, 0x5, 0x6, 0x7, 0x8, 0x9, 0xA, 0xB, 0xC, 0xD, 0xE, 0xF, # 1
                 0x0, 0x2, 0x4, 0x6, 0x8, 0xA, 0xC, 0xE, 0x3, 0x1, 0x7, 0x5, 0xB, 0x9, 0xF, 0xD, # 2
                 0x0, 0x3, 0x6, 0x5, 0xC, 0xF, 0xA, 0x9, 0xB, 0x8, 0xD, 0xE, 0x7, 0x4, 0x1, 0x2, # 3
                 0x0, 0x4, 0x8, 0xC, 0x3, 0x7, 0xB, 0xF, 0x6, 0x2, 0xE, 0xA, 0x5, 0x1, 0xD, 0x9, # 4
                 0x0, 0x5, 0xA, 0xF, 0x7, 0x2, 0xD, 0x8, 0xE, 0xB, 0x4, 0x1, 0x9, 0xC, 0x3, 0x6, # 5
                 0x0, 0x6, 0xC, 0xA, 0xB, 0xD, 0x7, 0x1, 0x5, 0x3, 0x9, 0xF, 0xE, 0x8, 0x2, 0x4, # 6
                 0x0, 0x7, 0xE, 0x9, 0xF, 0x8, 0x1, 0x6, 0xD, 0xA, 0x3, 0x4, 0x2, 0x5, 0xC, 0xB, # 7
                 0x0, 0x8, 0x3, 0xB, 0x6, 0xE, 0x5, 0xD, 0xC, 0x4, 0xF, 0x7, 0xA, 0x2, 0x9, 0x1, # 8
                 0x0, 0x9, 0x1, 0x8, 0x2, 0xB, 0x3, 0xA, 0x4, 0xD, 0x5, 0xC, 0x6, 0xF, 0x7, 0xE, # 9
                 0x0, 0xA, 0x7, 0xD, 0xE, 0x4, 0x9, 0x3, 0xF, 0x5, 0x8, 0x2, 0x1, 0xB, 0x6, 0xC, # A
                 0x0, 0xB, 0x5, 0xE, 0xA, 0x1, 0xF, 0x4, 0x7, 0xC, 0x2, 0x9, 0xD, 0x6, 0x8, 0x3, # B
                 0x0, 0xC, 0xB, 0x7, 0x5, 0x9, 0xE, 0x2, 0xA, 0x6, 0x1, 0xD, 0xF, 0x3, 0x4, 0x8, # C
                 0x0, 0xD, 0x9, 0x4, 0x1, 0xC, 0x8, 0x5, 0x2, 0xF, 0xB, 0x6, 0x3, 0xE, 0xA, 0x7, # D
                 0x0, 0xE, 0xF, 0x1, 0xD, 0x3, 0x2, 0xC, 0x9, 0x7, 0x6, 0x8, 0x4, 0xA, 0xB, 0x5, # E
                 0x0, 0xF, 0xD, 0x2, 0x9, 0x6, 0x4, 0xB, 0x1, 0xE, 0xC, 0x3, 0x8, 0x7, 0x5, 0xA ]# F

# AUT64 table de substitution                                  
table_sub = [0x0, 0x1, 0x9, 0xE, 0xD, 0xB, 0x7, 0x6, 0xF, 0x2, 0xC, 0x5, 0xA, 0x4, 0x3, 0x8]

#AUT64 table TD utilisé pour reconstituer la clé KG
table_td=[0x7D, 0x56, 0x99, 0x65, 0x8C, 0x74, 0x82, 0x83,
          0x9B, 0x92, 0x7B, 0xA1, 0xAA, 0xB0, 0x64, 0xCF,
          0xB9, 0xDE, 0x5D, 0xED, 0xC8, 0xFC, 0x46, 0x0B,
          0xD7, 0x1A, 0x3F, 0x29, 0xC6, 0x38, 0x28, 0x47]


On peut maintenant définir la fonction de compression

In [15]:
def compress(state, key_reg, roundN):
    """
    Réalise une compression sur les données 'state' en utilisant la clé 'key_reg' pour le tour 'roundN'.

    Args:
        state (list[int]): Liste des 8 octets d'entrée (état) à compresser.
        key_reg (list[int]): Liste des valeurs de la clé de registre.
        roundN (int): Numéro du tour en cours.

    Returns:
        int: Résultat de la compression sous la forme d'un octet.

    Description:
        Cette fonction réalise une compression sur les 8 premiers octets de la liste 'state' en utilisant les
        valeurs du registre de clé 'key_reg' pour le tour spécifié 'roundN'. La compression est effectuée en
        combinant les nibbles inférieurs et supérieurs de chaque octet d'état avec les nibbles de clé
        correspondants. Ensuite, la fonction de compression 'table_offset' est appliquée à ces combinaisons et
        le résultat est XORé avec les variables 'r5' et 'r6' pour obtenir le résultat final de la compression.

        La dernière étape du calcul concerne le dernier octet de la liste 'state', où le processus est légèrement
        différent en raison de la substitution des nibbles de clé et du déplacement des résultats.

        Le résultat final de la compression est renvoyé sous la forme d'un octet unique.

    """

    r5 = 0
    r6 = 0
    for byte in range(0, 7):# Calcul sur les 7 premiers octets
        ## Nibble inférieur dans l'octet ## 
        ln = state[byte] & 0xf # Obtention du nibble inférieur de l'octet
        lk = key_reg[table_ln[8*roundN + byte]] # Obtention du nibble de clé
        p0 = ln | ((lk << 4) & 0xf0)  # Combinaison du nibble d'état et du nibble de clé  
        r6 ^= table_offset[p0]# Application de la fonction de compression table_offset à p0 et XOR avec r6

        
       ## Nibble supérieur dans l'octet ##
        un = (state[byte] >> 4) & 0xf  # Obtention du nibble supérieur de l'octet
        uk = key_reg[table_un[8*roundN + byte]] # Obtention du nibble de clé  
        p1 = un | ((uk << 4) & 0xf0) # Combinaison du nibble d'état et du nibble de clé uk,un
        r5 ^= table_offset[p1] # Application de la fonction de compression table_offset à p1 et XOR avec r5
    ## Calcul du dernier octet (7)
    
    ## Nibble inférieur dans l'octet ##
    ln = state[7] & 0xf # Obtention du nibble inférieur de l'octet
    lk = key_reg[table_ln[8*roundN + 7]] # Obtention du nibble de clé 
    ls = (table_sub[lk] << 4) & 0xf0     # Substitution et déplacement du nibble inférieur vers le nibble supérieur

    # Recherche de l'index de ln dans table_offset à partir de ls + (0, 1, 2,..., 15)

    for i_ln in range(0,16):
        if table_offset[ls + i_ln] == ln:
            break
    r6 ^= i_ln # XOR avec r6

    ## Nibble supérieur dans l'octet ##
    un = (state[7] >> 4) & 0xf # Obtention du nibble supérieur de l'octet
    uk = key_reg[table_un[8*roundN + 7]] # Obtention du nibble de clé   
    us = (table_sub[uk] << 4) & 0xf0    # Substitution et déplacement du nibble inférieur vers le nibble supérieur

   # Recherche de l'index de un dans table_offset à partir de us + (0, 1, 2,..., 15)
    for i_un in range(0,16):
        if table_offset[us + i_un] == un:
            break
    r5 ^= i_un # XOR avec r5
    
    result = ((r5 << 4) & 0xf0) | (r6 & 0x0f) #r5,r6
            
    return result


In [16]:
compress([1,2,3,4,5,6,7,8],[0,1,2,3,4,5,6,7],7)

5

Avant d'implémenter l'algorithme de chiffrement de données, on doit d'abord définir les S-Box et P-Box


In [17]:
def apply_sbox(byte, sbox):
    """
    Applique la substitution 4x4 'sbox' sur l'octet 'byte' de manière nibble par nibble.

    Args:
        byte (int): Octet (8 bits) sur lequel appliquer la substitution.
        sbox (list[int]): Liste représentant la table de substitution 4x4 (S-box).

    Returns:
        int: Résultat sous forme d'entier 8 bits après l'application de la substitution.

    Description:
        Cette fonction applique la substitution 'sbox' sur l'octet 'byte' en prenant en compte les nibbles
        inférieur et supérieur séparément. Le résultat final est un entier 8 bits qui représente le
        résultat complet après l'application de la substitution.
    """
    result = 0
    
    # Nibble inférieur
    result = sbox[byte & 0xf]
    # Nibble supérieur
    result = result | sbox[(byte >> 4) & 0x0f] << 4
    
    return result


def apply_pbox(input_data, pbox, n):
    """
    Applique la permutation 'pbox' aux données d'entrée 'input_data' 'n' fois.

    Args:
        input_data (list[int]): Liste des octets d'entrée sur lesquels appliquer la permutation.
        pbox (list[int]): Liste représentant la table de permutation (P-box).
        n (int): Nombre de fois à appliquer la permutation.

    Returns:
        list[int]: Liste des octets de sortie après l'application de la permutation.

    Description:
        Cette fonction applique la permutation 'pbox' aux données d'entrée 'input_data' 'n' fois. Elle
        effectue la permutation sur chaque octet d'entrée et renvoie la liste des octets de sortie après
        les 'n' applications de la permutation.
    """
    input_data_copy = copy.copy(input_data)

    for rnd in range(0, n):
        output_data = [0] * 8
        for bIdx, byte in enumerate(input_data_copy):
            output_data[pbox[bIdx]] = input_data_copy[bIdx]        
        input_data_copy = copy.copy(output_data)
        
    return output_data


def apply_pbox_bitwise(input_data, pbox):
    """
    Applique la permutation 'pbox' aux données d'entrée 'input_data' au niveau des bits.

    Args:
        input_data (int): Octet d'entrée (8 bits) sur lequel appliquer la permutation.
        pbox (list[int]): Liste représentant la table de permutation (P-box).

    Returns:
        int: Entier 8 bits représentant les données d'entrée après l'application de la permutation.

    Description:
        Cette fonction applique la permutation 'pbox' aux données d'entrée 'input_data' bit par bit. Elle
        effectue la permutation en utilisant les positions définies dans la table 'pbox' et renvoie le
        résultat final sous forme d'un entier 8 bits.
    """
    result = 0

    for bit in range(8):
        if (input_data & (1 << bit)):
            result |= (1 << pbox[bit])

    return result


In [18]:
apply_pbox_bitwise(78,[0,1,2,3,4,5,6,7])

78

Finalement, on définit la fonction qui chiffre les donées

In [20]:
def encrypt(state, key_reg, pbox, sbox, nRounds):
    """
    Effectue le processus de chiffrement sur l'état 'state' en utilisant la clé 'key_reg' et les tables de permutation et substitution.

    Args:
        state (list[int]): Liste des octets d'entrée (état) à chiffrer.
        key_reg (list[int]): Liste des valeurs du registre de clé.
        pbox (list[int]): Liste représentant la table de permutation (P-box).
        sbox (list[int]): Liste représentant la table de substitution 4x4 (S-box).
        nRounds (int): Nombre de tours de chiffrement à effectuer.

    Returns:
        list[int]: Liste des octets chiffrés après 'nRounds' tours.

    Description:
        Cette fonction effectue le chiffrement de l'état 'state' en utilisant la clé 'key_reg' et les tables
        de permutation 'pbox' et substitution 'sbox'. Elle répète le processus de chiffrement 'nRounds' fois.
        Pour chaque tour de chiffrement, la permutation 'pbox' est appliquée une fois sur l'état interne,
        suivie des étapes de compression, substitution et permutation bit à bit. Le résultat final est
        obtenu après 'nRounds' tours de chiffrement et est renvoyé sous forme d'une liste des octets
        chiffrés.
    """
    stateInternal = copy.deepcopy(state)

    for roundN in range(0, nRounds):
        stateInternal = apply_pbox(stateInternal, pbox, 1)
        step1 = compress(stateInternal, key_reg, roundN)
        step2 = list(bin(apply_sbox(step1, sbox))[2:].zfill(8))
        step2 = int(''.join(step2), 2)
        step3 = apply_pbox_bitwise(step2, pbox)

        step4 = int(apply_sbox(step3, sbox))
        stateInternal[7] = step4

    return stateInternal


Voici un exemple d'exécution de l'algorithme AUT64

In [78]:
key_reg = [12, 6, 4, 14, 5, 4, 8, 11]
state= [0x1,0x2,0x3,0x4,0x5,0x6,0x7,0x8]
pbox = [2, 6, 0, 5, 7, 4, 3, 1]
sbox=[0, 1, 2, 3, 9, 12, 11, 5, 14, 7, 4, 10, 6, 8, 13, 15]
nRounds=24
print(encrypt(state,key_reg,pbox,sbox,nRounds))


[1, 194, 3, 7, 168, 29, 252, 132]


Maintenant qu'on a bien implémenté le système, on passe à l'attaque ! Comme expliqué dans le document lié à ce notebook, l'attaque consiste à récupérer les 3 différentes clés

Pour expliquer brièvement l'idée de l'attaque, nous demandons d'abord à l'utilisateur d'insérer les valeurs de ID0, ID1, ID2 et ID3 qu'il a récupérées lors de sa connexion avec le transpondeur. Cela nous permettra de reconstituer la clé KG.

En ce qui concerne la clé de permutation Kt, les auteurs ont démontré que le schéma de gestion de clé AUT64 réduit l'espace des clés de permutation à seulement 16 clés par fabricant de véhicules. Cela signifie qu'en lisant la clé Kt à partir de deux unités d'immobilisation différentes et en identifiant la partie constante, nous n'aurons que 16 clés candidates.

Quant à la clé de substitution Kp, elle a une taille de 16! = 44.3 bits. Pour la retrouver, il suffit de faire une recherche exhaustive en testant toutes les combinaisons possibles. Plutôt que d'utiliser deux boucles imbriquées (une pour Kp et l'autre pour Kt), il est plus efficace de concevoir un algorithme ciblant uniquement la clé Kp et ayant Kt comme l'un de ses paramètres. Ainsi, dans le pire des cas, il suffit de lancer l'algorithme 16 fois pour trouver la clé Kp.

Avant d'implémenter l'attaque, on définit d'abord un algorithme qui demande de saisir les 16 clés candidates kt

In [63]:
def demander_liste():
    liste_principale = []

    for i in range(16):
        liste_saisie = input(f"Insérez la sous-liste {i+1} (8 éléments séparés par des espaces) : ")
        sous_liste = [int(element) for element in liste_saisie.split()]
        
        # Vérifier que la sous-liste a exactement 8 éléments
        if len(sous_liste) != 8:
            print("La sous-liste doit avoir exactement 8 éléments. Veuillez réessayer.")
            i -= 1
            continue

        # Vérifier que les éléments sont compris entre 0 et 7
        if not all(0 <= element <= 7 for element in sous_liste):
            print("Les éléments doivent être des entiers entre 0 et 7. Veuillez réessayer.")
            i -= 1
            continue

        liste_principale.append(sous_liste)

    return liste_principale
kt=demander_liste()

Une fois que l'utilisateur a saisi la liste des 16 clés candidates, on peut définir notre attaque

In [66]:
def BruteForce(P, C, kt, nRounds):
    """
    Fonction pour effectuer une attaque par force brute sur AUT64.

    Parameters:
        P (list): Liste représentant le plaintext (8 éléments de 4 bits chacun).
        C (list): Liste représentant le chiffré (8 éléments de 4 bits chacun).
        kt (list): Liste représentant la clé de permutation Kt (8 éléments de 4 bits chacun).
        nRounds (int): Le nombre de tours à utiliser dans l'algorithme de chiffrement.

    Returns:
        list: Une liste contenant les résultats de l'attaque : [KG, kt, kp].
              KG : Liste représentant la clé KG (8 éléments de 4 bits chacun).
              kt : Liste représentant la clé de permutation Kt (8 éléments de 4 bits chacun).
              kp : Liste représentant la clé de substitution Kp (16 éléments de 4 bits chacun).
        None: Si l'attaque n'a pas réussi à trouver les clés.
    """

    # Récupération des éléments nécessaires pour le calcul de KG
    ID0 = int(input("Veuillez insérer la valeur de ID0 : "))
    ID1 = int(input("Veuillez insérer la valeur de ID1 : "))
    ID2 = int(input("Veuillez insérer la valeur de ID2 : "))
    ID3 = int(input("Veuillez insérer la valeur de ID3 : "))
    u = (ID0 & 0xE) << 1
    kG3 = ID3 ^ TD[3 + u]
    kG2 = ID2 ^ TD[2 + u] ^ kG3
    kG1 = ID1 ^ TD[1 + u] ^ kG3
    kG0 = ID0 ^ TD[u] ^ kG3
    KG = [kG0 >> 4, kG0 & 0xF, kG1 >> 4, kG1 & 0xF, kG2 >> 4, kG2 & 0xF, kG3 >> 4, kG3 & 0xF]

    kp = []
    # Toutes les combinaisons possibles de la sbox (kp)
    sbox = permutations([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])
    for kp1 in sbox:
        C_prime = encrypt(P, KG, kt, kp1, nRounds)
        if C_prime == C:
            kp = kp1
            return [KG, kt, list(kp)]

    return None

On peut maintenant exécuter l'algorithme d'attaque sur chaque clé kt candidate et s'arréter lorsque la bonne clé est retrouvée

In [67]:
def final(P, C, nRounds):
    """
    Fonction pour chercher les clés KG, Kt et Kp en utilisant la fonction BruteForce.

    Parameters:
        P (list): Liste représentant le plaintext (8 éléments de 4 bits chacun).
        C (list): Liste représentant le chiffré (8 éléments de 4 bits chacun).
        nRounds (int): Le nombre de tours à utiliser dans l'algorithme de chiffrement.

    Returns:
        list: Une liste contenant les résultats des clés trouvées : [KG, Kt, Kp].
              KG : Liste représentant la clé KG (8 éléments de 4 bits chacun).
              Kt : Liste représentant la clé de permutation Kt (8 éléments de 4 bits chacun).
              Kp : Liste représentant la clé de substitution Kp (16 éléments de 4 bits chacun).
        None: Si les clés n'ont pas été trouvées.
    """

    s = []
    for element in kt:
        # Chercher les clés KG, Kt et Kp avec la fonction BruteForce
        cand = BruteForce(P, C, element, nRounds)
        if cand is not None:
            return cand

    return None


On peut tester la validité de l'attaque, en fixant la valeur de la clé KG et kt (qui sont censées à ne pas être difficile à récueprer)

In [80]:
def test_validité(P,C,nRounds):
    kp = []
    # Toutes les combinaisons possibles de la sbox (kp)
    sbox = permutations([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])
    for kp1 in sbox:
        C_prime = encrypt(P, KG, kt, kp1, nRounds)
        if C_prime == C:
            kp = kp1
            return [KG, kt, list(kp)]

    return None
KG=[12, 6, 4, 14, 5, 4, 8, 11]
P=[0x1,0x2,0x3,0x4,0x5,0x6,0x7,0x8]
C=[1, 194, 3, 7, 168, 29, 252, 132]
kt=[2, 6, 0, 5, 7, 4, 3, 1]
kp=[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 11, 15, 9]
nRounds=24
C=encrypt(P,KG,kt,kp,nRounds)
test_validité(P,C,nRounds)

[[12, 6, 4, 14, 5, 4, 8, 11],
 [2, 6, 0, 5, 7, 4, 3, 1],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 11, 15, 9]]